In [ ]:

import numpy as np
import pandas as pd
from sklearn import preprocessing
import csv
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [ ]:
dataset = np.loadtxt("pima-indians-diabetes.csv", delimiter=",")
X = dataset[:,0:8]
Y = dataset[:,8]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2,random_state=4)

In [ ]:
Y_train = Y_train.reshape(614,1)
Y_test = Y_test.reshape(154,1)

In [ ]:
# First Neural Network with normal backpropagation implementation

class NeuralNetwork(object):
    
    def __init__(self):
        self.input = X_train.shape[1]
        self.learning = 0.04
        self.hidden = (X_train.shape[1]*2)
        self.output = 1
    
        self.W1 = np.random.randn(self.input, self.hidden)
        self.W2 = np.random.randn(self.hidden, self.output)
    
    def sigmoid(self,x):
        return 1.0/(1+ np.exp(-x))
    
    def sigmoid_derivative(self,x):
        return x * (1 - x)
    
    def feedforward(self,X):
        self.z = np.dot(X, self.W1)
        self.z2 = self.sigmoid(self.z)
        self.z3 = np.dot(self.z2, self.W2)
        O = self.sigmoid(self.z3)
        return O
    
    def backpropgation(self,X, Y, O):
        self.o_error = Y - O
        self.o_delta = self.o_error*self.sigmoid_derivative(O)

        self.z2_error = self.o_delta.dot(self.W2.T) 
        self.z2_delta = self.z2_error*self.sigmoid_derivative(self.z2)

        self.W1 += self.learning*X.T.dot(self.z2_delta)
        self.W2 += self.learning*self.z2.T.dot(self.o_delta)
        
    def trainNN (self, X, Y):
        O = self.feedforward(X)
        self.backpropgation(X, Y, O)
        
    def accuracy(self,X):
        self.z = np.dot(X, self.W1)
        self.z2 = self.sigmoid(self.z) 
        self.z3 = np.dot(self.z2, self.W2)
        O = self.sigmoid(self.z3) 
        return O


In [ ]:
# Train first neural network

NN = NeuralNetwork()

for i in range(250):
    print ("Loss: \n" + str(np.mean(np.square(Y_train- NN.feedforward(X_train)))))
    NN.trainNN(X_train, Y_train)

In [ ]:
# Second neural network with modified backpropagation

class NeuralNetwork2(object):
    
    def __init__(self):
        self.input = X_train.shape[1]
        self.learning = 0.04
        self.hidden = (X_train.shape[1]*2)
        self.output = 1
    
        self.W1 = np.random.randn(self.input, self.hidden)
        self.W2 = np.random.randn(self.hidden, self.output)

    def feedforward(self,X):
        self.z = np.dot(X, self.W1)
        self.z2 = self.sigmoid(self.z)
        self.z3 = np.dot(self.z2, self.W2)
        O = self.sigmoid(self.z3)
        return O

    def sigmoid(self,x):
        return 1.0/(1+ np.exp(-x))
    
    
    def sigmoid_derivative(self,x):
        return x * (1 - x)

    
    def backpropgation(self,X, Y, O):

        for fl in O > 0.5:
            if fl:
                self.o_error = 1-O
            else:
                self.o_error = Y-O
                
        self.o_delta = self.o_error*self.sigmoid_derivative(O)

        self.z2_error = self.o_delta.dot(self.W2.T) 
        self.z2_delta = self.z2_error*self.sigmoid_derivative(self.z2)

        self.W1 += self.learning*X.T.dot(self.z2_delta) 
        self.W2 += self.learning*self.z2.T.dot(self.o_delta) 
    
    def train (self, X, Y):
        O = self.feedforward(X)
        self.backpropgation(X, Y, O)
        
    def accuracy(self,X):
        self.z = np.dot(X, self.W1) 
        self.z2 = self.sigmoid(self.z)
        self.z3 = np.dot(self.z2, self.W2) 
        O = self.sigmoid(self.z3) 
        return O

In [ ]:
# Train second neural network

NN2 = NeuralNetwork2()
for i in range(250): 
    print ("Loss: \n" + str(np.mean(np.square(Y_train- NN2.feedforward(X_train)))))
    print ("\n")
    NN2.train(X_train, Y_train)

In [ ]:
# First Neural Network with normal backpropagation 

NN = NeuralNetwork()
y_pred1  = NN.accuracy(X_test)
y_pred1 = [round(x[0]) for x in y_pred1 ]

cm = confusion_matrix(Y_test,y_pred1)
FPR = cm[0,1]/(cm[0,1] + cm[0,0])
FNR = cm[1,0]/(cm[1,0] + cm[1,1])

print("Output with normal implementation of backpropagation:")
print("\nConfusion Matrix:\n")
print(cm)

print("\nFalse Positive Rate (FPR) :- %.2f%%" % (FPR * 100))
print("False Negative Rate (FNR) :- %.2f%%" % (FNR * 100))

In [ ]:
# Second Neural Network with updated backpropagation 

NN2 = NeuralNetwork2()
y_pred2  = NN2.accuracy(X_test)
y_pred2 = [round(x[0]) for x in y_pred2 ]

cm1 = confusion_matrix(Y_test,y_pred2)
FPR = cm1[0,1]/(cm1[0,1] + cm1[0,0])
FNR = cm1[1,0]/(cm1[1,0] + cm1[1,1])

print("Output after modification of backpropagation:")
print("\nConfusion Matrix:\n")
print(cm1)

print("\nFalse Positive Rate (FPR) :- %.2f%%" % (FPR * 100))
print("False Negative Rate (FNR) :- %.2f%%" % (FNR * 100))